In [1]:
import pandas as pd

models = ['answer_gpt4', 'answer_gpt35']
reviewers = ['gpt-3.5-turbo-0301', 'gpt-4']

def review_filename(modelA, modelB, reviewer):
    return f'{modelA}-vs-{modelB}-{reviewer}-reviewer-threeclass.jsonl'

df = pd.read_json('answer_gpt4_vs_answer_gpt35_gpt-3.5-turbo-0301_reviewer_threeclass.jsonl', lines=True)
df.head()

,review_id,question_id,answer1_id,answer2_id,reviewer_id,metadata,text,score
0,4gLzStHWqR2s33juPh4UuS,1,k3KTH9U8v39Sqqb2Z4jo8C,BZGowHM7L3RvtWRktKZjLT,1,{},Assistant 1 provided a comprehensive and detai...,1
1,XrmReMXYKSMN4fDsjpiAwz,2,asuUCLx4qNGr4TgeHVDoxe,GsrPLfiNEcRDYAC5aUoY23,1,{},Assistant 1's answer is very detailed and prov...,1
2,VR9QTtbrTHXmY2ngvur8sn,3,iekZGGkvTnwm5T4J6MtvDi,5SGfexmPoJTWoVbPMB87KB,1,{},Assistant 1 provided a more comprehensive and ...,1
3,EmE9XALN58re3tgxLgsNMu,4,eo5mhtwgZYFrqp4wXrR3Z6,RcqQg364kgCkYPshqvmGSp,1,{},Assistant 1's answer is very comprehensive and...,1
4,GdqwTrZFoFRPppANNyv7sj,5,H6mF8revhnrFJ7XGJfD6pn,3R9jAndzLHQqJwmMi5mNox,1,{},"Assistant 1's answer is very helpful, relevant...",1


In [5]:
df[df.score == 2]

,review_id,question_id,answer1_id,answer2_id,reviewer_id,metadata,text,score
23,RyRqVHiCttPAFxnVAVgav8,24,3fQtwU9RCzYkrBm6EGJE9D,oNULT72cYUvit7D9SHb5aM,1,{},Assistant 1:\n- Helpfulness: 4/5\n- Relevance:...,2
45,hAqrN3Z6t3FbdBUTXjomVv,46,YC335Wet5qRAprLo7PjPKR,cU3wut3Ta3ySbRHGxfwgjc,1,{},Assistant 1:\nHelpfulness: 4/5\nRelevance: 4/5...,2
60,bYGFMQMLieGfJ3ZKo6DmCM,61,Y5tDAydR2v4e5X5PJCuJwE,2PRjT7j3V6487xZREfQfuD,2,{},Both Assistant 1 and Assistant 2 have provided...,2
66,mCoZbP5Tatfbaig8mJjiwe,67,TkoomxuTjCK3REYQ6X7WSD,NKXTwB3WtfFcmQZbTxBP5o,2,{},Both Assistant 1 and Assistant 2 have provided...,2


In [3]:
score_remap = {
    1: -1,
    2: 1,
    3: 0
}

0    -1
1    -1
2    -1
3    -1
4    -1
     ..
75   -1
76   -1
77   -1
78   -1
79   -1
Name: score, Length: 80, dtype: int64